<h1>ML Variance Model that predicts daily volatility of stock using linear regression</h1>

gathering data from yfinance


In [4]:
import yfinance as yf
import numpy as np
import matplotlib as plt

ticker = yf.Ticker('AAPL')
data = yf.download('AAPL', start='2022-01-01', end='2024-01-01') #input desired stock ticker here
volume = ticker.info["averageVolume10days"]

data.head()


[*********************100%***********************]  1 of 1 completed

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2022-01-03,179.076599,182.009995,182.880005,177.710007,177.830002,104487900
2022-01-04,176.803833,179.699997,182.940002,179.119995,182.630005,99310400
2022-01-05,172.100861,174.919998,180.169998,174.639999,179.610001,94537600
2022-01-06,169.227936,172.000000,175.300003,171.639999,172.699997,96904000
2022-01-07,169.395187,172.169998,174.139999,171.029999,172.889999,86709100


</h2>calculate daily returns and volatility</h2>

In [ ]:
# Calculate daily returns (returns list of percentage change for each day compared to previous)
data['Return'] = data['Adj Close'].pct_change()

# Calculate rolling variance (volatility) over a 20-day window
data['Volatility'] = data['Return'].rolling(window=20).var()

# Drop rows with NaN values resulting from calculations
data.dropna(inplace=True)

# Display the updated data
data[['Return', 'Volatility']].head()

use previous day's volatility and moving average as features

In [ ]:
# Feature 1: Previous day's volatility
data['Volatility_Lag1'] = data['Volatility'].shift(1)

# Feature 2: 20-day moving average of returns
data['MA_Return'] = data['Return'].rolling(window=20).mean()

# Drop any new NaN values (i.e. empty values)
data.dropna(inplace=True)

# Define feature matrix X and target vector y
X = data[['Volatility_Lag1', 'MA_Return']]
y = data['Volatility']

split data (use 80 20 ratio for training and testing)

In [ ]:
from sklearn.model_selection import train_test_split

# Since it's time series, we'll split based on index
split_ratio = 0.8
split_index = int(len(X) * split_ratio)

X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]
